In [2]:
! pip install -q pydantic groq

In [3]:
from enum import Enum
import json
import requests
from groq import Groq
from pydantic import BaseModel, Field

In [4]:
from google.colab import userdata

groq_api_key = userdata.get('GROQ_API_KEY')

In [5]:
client = Groq(api_key=groq_api_key)
MODEL = "openai/gpt-oss-20b"

##### Json Schema

In [6]:
response = client.chat.completions.create(
  model = MODEL,
  messages = [
    {
      "role": "system",
      "content": "You are a helpful math tutor."
    },
    {
      "role": "user",
      "content": "solve 8x + 31 = 2"
    }
  ],
  response_format = {
    "type": "json_schema",
    "json_schema": {
      "name": "math_response",
      "strict": True,
      "schema": {
        "type": "object",
        "properties": {
          "steps": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "explanation": {
                  "type": "string"
                },
                "output": {
                  "type": "string"
                }
              },
              "required": ["explanation", "output"],
              "additionalProperties": False
            }
          },
          "final_answer": {
            "type": "string"
          }
        },
        "required": ["steps", "final_answer"],
        "additionalProperties": False
      }
    }
  }
)

print(response.choices[0].message.content)

{"steps":[{"explanation":"Subtract 31 from both sides to isolate the term with x.","output":"8x = -29"},{"explanation":"Divide both sides by 8 to solve for x.","output":"x = -29/8"}],"final_answer":"x = -29/8"}


In [7]:
answer =  response.choices[0].message.content
parsed_answer = json.loads(answer)
steps = parsed_answer['steps']
final_answer = parsed_answer['final_answer']

In [8]:
final_answer

'x = -29/8'

In [9]:
for step in steps:
    print(f"Explanation: {step['explanation']}")
    print(f"Output: {step['output']}")
    print()


Explanation: Subtract 31 from both sides to isolate the term with x.
Output: 8x = -29

Explanation: Divide both sides by 8 to solve for x.
Output: x = -29/8



##### Pydantic

In [10]:
query = "solve 8x + 31 = 2"

In [11]:
system_prompt = "You are a helpful math tutor."

In [24]:
class MathTutor(BaseModel):
    class Step(BaseModel):
        explanation: str = Field(description="Description of the step taken.")
        output: str = Field(description="Only answer of each step without text")

    steps: list[Step]
    final_answer: str = Field(
        description="The final answer of the equation."
    )


def math_response(query: str):
    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
        # response_format=MathTutor,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "math_tutor_schema",
                "schema": MathTutor.model_json_schema(),
            },
        },
    )

    # Parse JSON back into Pydantic model
    content = completion.choices[0].message.content
    return MathTutor.model_validate_json(content)


In [25]:
response_pydantic = math_response(query)
print(response_pydantic.model_dump())

{'steps': [{'explanation': 'Subtract 31 from both sides to isolate the term with x.', 'output': '-29'}, {'explanation': 'Divide both sides by 8 to solve for x.', 'output': '-29/8'}], 'final_answer': '-29/8'}
